In [ ]:
#배치처리 시 시퀀스 길이가 다를때 어떻게 해야하는지
#디코더의 인풋으로는 정확히 무엇을 넣어야 하고
#아웃풋과 어떻게 loss계산을 해야하는지
#배치속에서 시퀀스 길이가 다를 때 추가적인 마스크를 써주어야 하는데 어떤 모양이여야 하는지
#패딩은 예측해야 하는지(손실값 계산)
#예측 안 한다면 손실값에 관여 안하도록 어떻게 해야할지

In [ ]:
#하고있던거: 0, 1, 2(eos), 3(sos) 맞춤으로 된 트랜스포머 구조 바꾸기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from glob import glob

In [3]:
data = glob('/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/*.xlsx')

In [4]:
data

['/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/1_구어체(2).xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/2_대화체.xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/5_문어체_조례.xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/1_구어체(1).xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/4_문어체_한국문화.xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/6_문어체_지자체웹사이트.xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(2).xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(4).xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(3).xlsx',
 '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(1).xlsx']

In [ ]:
df = pd.DataFrame(columns = ['원문','번역문'])
path = '/content/drive/MyDrive/kor-eng/한국어-영어 번역(병렬) 말뭉치/'

file_list = [ '1_구어체(2).xlsx',
 '2_대화체.xlsx',
 '5_문어체_조례.xlsx',
 '1_구어체(1).xlsx',
 '4_문어체_한국문화.xlsx',
 '6_문어체_지자체웹사이트.xlsx',
 '3_문어체_뉴스(4).xlsx',
 '3_문어체_뉴스(3).xlsx',
 '3_문어체_뉴스(2).xlsx',
 '3_문어체_뉴스(1).xlsx']

for data in file_list:
    temp = pd.read_excel(path+data)
    df = pd.concat([df,temp[['원문','번역문']]])

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
!pip install konlpy

In [ ]:
!pip install nltk

In [ ]:
from konlpy.tag import Okt
import nltk
nltk.download('popular')
from nltk.tokenize import word_tokenize

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import math
from sklearn.preprocessing import LabelEncoder
kor_encoder = LabelEncoder()
eng_encoder = LabelEncoder()

In [ ]:
a = df.to_numpy()
a.shape

In [ ]:
"""
idx = np.arange(a.shape[0])
np.random.shuffle(idx)


a = a[idx]
"""

'\nidx = np.arange(a.shape[0])\nnp.random.shuffle(idx)\n\n\na = a[idx]\n'

In [ ]:
print(a[3,0])
print(a[3,1])

In [ ]:
tokenizer = Okt()
print(tokenizer.morphs(a[0,0]))

In [ ]:
word_tokenize(a[0,1])

In [ ]:
kor = []
eng = []
for i in range(150000):
  kor.append(tokenizer.morphs(a[i,0]))
for i in range(150000):
  eng.append(word_tokenize(a[i,1]))

In [ ]:
print(len(kor))
print(len(eng))

In [ ]:
kor[0]

In [ ]:
#토큰화->배치처리('[PAD]'처리등과 함께)  (배열속 넘파이의 구조)   ->       (텐서 변환 후) (학습 함수 안에서)라벨인코딩   -> loss계산할때 원-핫 변환

In [ ]:
kor_unq = ['[PAD]','[SOS]','[EOS]']
for i in range(150000):
  for j in range(len(kor[i])):
    if kor[i][j] not in kor_unq:
      kor_unq.append(kor[i][j])
kor_unq = np.array(kor_unq)
kor_encoder.fit(kor_unq)


In [ ]:
eng_unq = ['[PAD]','[SOS]','[EOS]']
for i in range(150000):
  for j in range(len(eng[i])):
    if eng[i][j] not in eng_unq:
      eng_unq.append(eng[i][j])
eng_unq = np.array(eng_unq)
eng_encoder.fit(eng_unq)

In [ ]:
kor_token_len = len(kor_encoder.classes_)
eng_token_len = len(eng_encoder.classes_)

In [ ]:
eng_token_len

In [ ]:
pad_index = eng_encoder.transform(['[PAD]'])[0]
sos_index = eng_encoder.transform(['[SOS]'])[0]

In [ ]:
pad_index < sos_index

In [ ]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12, batch_first=True)

In [ ]:
src = torch.rand((32, 32, 512))
tgt = torch.rand((32, 1, 512))
out = transformer_model(src, tgt)
out.shape

torch.Size([32, 1, 512])

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        self.dropout = nn.Dropout(dropout_p)

        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1)
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model)

        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)

        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)

    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])


In [ ]:
hi = nn.Embedding(100, 100)
hihi = hi(torch.tensor([1,2,3,4,5,6,7,8,9,10]))
hihi.shape

torch.Size([10, 100])

In [ ]:
math.sqrt(256)

16.0

In [ ]:
class Transformer(nn.Module):
    def __init__( self, kor_token_len, eng_token_len, dim_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p, ):
        super().__init__()
        self.dim_model = dim_model
        self.positional_encoder = PositionalEncoding(dim_model=dim_model, dropout_p=dropout_p, max_len=5000)
        self.kor_embedding = nn.Embedding(kor_token_len, dim_model)
        self.eng_embedding = nn.Embedding(eng_token_len, dim_model)
        self.transformer = nn.Transformer(
            batch_first=True,
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
        )
        self.out = nn.Linear(dim_model, eng_token_len-2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):
        src = self.kor_embedding(src) * math.sqrt(self.dim_model)
        tgt = self.eng_embedding(tgt) * math.sqrt(self.dim_model)
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)

        transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        out = self.sigmoid(self.out(transformer_out))
        return out

    def get_tgt_mask(self, size) -> torch.tensor:
        mask = torch.tril(torch.ones(size, size) == 1)
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, float(0.0))

        return mask

    def create_pad_mask(self, matrix: torch.tensor) -> torch.tensor:
        return (matrix == '[PAD]' or matrix == '[EOS]')

In [ ]:
def batchify_data(data, batch_size=16, padding=False):
    batches = []
    for idx in range(0, len(data), batch_size):
        if idx + batch_size < len(data):
            if padding:
                max_batch_length = 0
                for seq in data[idx : idx + batch_size]:
                    if len(seq) > max_batch_length:
                        max_batch_length = len(seq)
                for seq_idx in range(batch_size):
                    remaining_length = max_batch_length - len(data[idx + seq_idx])
                    data[idx + seq_idx] = ['[SOS]'] + data[idx + seq_idx] + ['[EOS]']
                    data[idx + seq_idx] += ['[PAD]'] * remaining_length

            batches.append(np.array(data[idx : idx + batch_size]))

    print(f"{len(batches)} batches of size {batch_size}")

    return batches

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Transformer(kor_token_len=kor_token_len, eng_token_len = eng_token_len, dim_model=256, num_heads=4, num_encoder_layers=6, num_decoder_layers=6, dropout_p=0.1).to(device)
opt = torch.optim.SGD(model.parameters(), lr=0.000001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:

model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/model.pt'))
opt.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/opt.pt'))


In [ ]:
def onehot(y, eng_token_len):
  a = []
  for i in range(y.shape[0]):
    b = []
    for j in range(y[i].shape[0]):
      c = [0] * eng_token_len
      c[y[i,j]] = 1
      b.append(c)
    a.append(b)
  a = torch.tensor(a, dtype=torch.float16)
  return a.to(device)

def train_loop(model, opt, loss_fn, kor_dataloader, eng_dataloader):
    model.train()
    total_loss = 0

    for i in range(len(kor_dataloader)):
        if i%50 == 0:
          print(i)
        X, y = kor_dataloader[i].copy(), eng_dataloader[i].copy()
        for j in range(len(X)):
          X[j] = kor_encoder.transform(X[j])
          y[j] = eng_encoder.transform(y[j])
        X = X.astype(np.int32)
        y = y.astype(np.int32)
        X, y = torch.tensor(X, dtype=torch.long, device=device), torch.tensor(y, dtype=torch.long, device=device)

        y_input = y[:,:-1]
        y_expected = y[:,1:]
        sequence_length = y_input.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length).to(device)

        pred = model(X, y_input, tgt_mask)
        y_expected = onehot(y_expected, eng_token_len)
        y_expected = torch.cat([y_expected[:,:,:pad_index], y_expected[:,:,pad_index+1:]],dim=2)
        loss = loss_fn(pred, y_expected)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss += loss.detach().item()

    return total_loss / len(kor_dataloader)

In [ ]:
def validation_loop(model, loss_fn, kor_dataloader, eng_dataloader):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for i in range(len(kor_dataloader)):
            X, y = kor_dataloader[i].copy(), eng_dataloader[i].copy()
            for j in range(len(X)):
              X[j].shape
              X[j] = kor_encoder.transform(X[j])
              y[j] = eng_encoder.transform(y[j])
            X = X.astype(np.int32)
            y = y.astype(np.int32)
            X, y = torch.tensor(X, dtype=torch.long, device=device), torch.tensor(y, dtype=torch.long, device=device)


            y_input = y[:,:-1]
            y_expected = y[:,1:]

            sequence_length = y_input.size(1)
            tgt_mask = model.get_tgt_mask(sequence_length).to(device)

            pred = model(X, y_input, tgt_mask)

            y_expected = onehot(y_expected, eng_token_len)
            y_expected = torch.cat([y_expected[:,:,:pad_index], y_expected[:,:,pad_index+1:]],dim=2)
            loss = loss_fn(pred, y_expected)
            total_loss += loss.detach().item()

    return total_loss / len(kor_dataloader)


In [ ]:
def fit(model, opt, loss_fn, epochs, kor_train_dataloader, eng_train_dataloader, kor_test_dataloader=0, eng_test_dataloader=0):
    train_loss_list, validation_loss_list = [], []
    min_loss = 10000
    print("Training and validating model")
    for epoch in range(epochs):
        print("-"*25, f"Epoch {epoch + 1}","-"*25)

        train_loss = train_loop(model, opt, loss_fn, kor_train_dataloader, eng_train_dataloader)
        train_loss_list += [train_loss]

        #validation_loss = validation_loop(model, loss_fn, kor_test_dataloader, eng_test_dataloader)
        #validation_loss_list += [validation_loss]

        print(f"Training loss: {train_loss:.4f}")
        #print(f"Validation loss: {validation_loss:.4f}")
        print()
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/model.pt')
        torch.save(opt.state_dict(), '/content/drive/MyDrive/Colab Notebooks/opt.pt')


    return train_loss_list, validation_loss_list

In [ ]:
def predict(model, input_sequence, max_length=15, SOS_token='[SOS]', EOS_token='[EOS]'):
    model.eval()
    save = input_sequence.copy()
    input_sequence[0] = kor_encoder.transform(input_sequence[0])
    input_sequence = input_sequence.astype(np.int32)
    X = torch.tensor(input_sequence, dtype=torch.long, device=device)

    y_input = eng_encoder.transform([SOS_token])
    y_input = y_input.astype(np.int32)
    y_input = torch.tensor(y_input, dtype=torch.long, device=device).reshape(1,1)

    for _ in range(max_length):
        tgt_mask = model.get_tgt_mask(y_input.size(1)).to(device)

        pred = model(X, y_input, tgt_mask)
        pred = pred.topk(1)[1].view(-1)[-1].item()
        if pred > eng_encoder.transform(['[PAD]'])[0]:
          pred = pred+1
        next_item = torch.tensor([[pred]], device=device)

        y_input = torch.cat((y_input, next_item), dim=1)

        if next_item.view(-1).item() == eng_encoder.transform(['[EOS]']):
          break
    print('------------------------------------------------------------------------------------')
    for i in range(save.shape[1]):
      print(save[0,i], end=" ")
    print()
    for i in range(y_input.shape[1]):
      print(eng_encoder.inverse_transform(y_input[0,i:i+1].cpu().numpy())[0], end=" ")
    print()
    print('-----------------------------------------------------------------------------------')




In [ ]:
for j in range(1,10):
  print('--------------------',j,'-----------------------')
  idx = list(range(150000))
  random.shuffle(idx)
  kkor = []
  eeng = []
  for i in range(150000):
      kkor.append(kor[idx[i]])
      eeng.append(eng[idx[i]])
  kor = kkor
  eng = eeng
  for i in range(5):
    print('--------------------',i,'-----------------------')
    kor_train_dataloader = batchify_data(kor[i*30000:(i+1)*30000], batch_size=64, padding=True)
    eng_train_dataloader = batchify_data(eng[i*30000:(i+1)*30000], batch_size=64, padding=True)
    train_loss_list, validation_loss_list = fit(model, opt, loss_fn, 1, kor_train_dataloader, eng_train_dataloader)
    for i in range(5):
      predict(model,kor_train_dataloader[0][i:i+1].copy(), max_length=30)

-------------------- 1 -----------------------
-------------------- 0 -----------------------
468 batches of size 64
468 batches of size 64
Training and validating model
------------------------- Epoch 1 -------------------------
0


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/model.pt')
torch.save(opt.state_dict(), '/content/drive/MyDrive/Colab Notebooks/opt.pt')

In [ ]:
for i in range(5):
  predict(model,kor_train_dataloader[2][i:i+1].copy(), max_length=30)

------------------------------------------------------------------------------------
[SOS] 저번 내한 때 는 콘서트 의 촬영 이 가능했었는데 , 올해 에는 촬영 이 불가능한가요 ? [EOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
[SOS] Kepco brochure Kepco Assassination brochure brochure brochure brochure brochure brochure brochure brochure brochure brochure brochure Aboriginal brochure Aboriginal brochure Aboriginal brochure Aboriginal brochure Aboriginal brochure Aboriginal brochure Aboriginal brochure Aboriginal 
-----------------------------------------------------------------------------------
------------------------------------------------------------------------------------
[SOS] 만약 그 들 이 내 공연 을 본다면 내 재능 에 놀랄 거야 . [EOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
[SOS] trickled trickled braveness FutureNet cultivate braveness aspirin cultivate braveness aspirin cultivate braveness cultivate braveness braveness cultivate cultivate braveness braveness cultivate